In [ ]:
from agents import Agent, Runner, function_tool, WebSearchTool, ModelSettings
from pydantic import BaseModel, Field

In [ ]:
class WebSearchItem(BaseModel):
    query: str = Field(description="The search term to use for the web search")
    reason: str = Field(description="The reason for the search")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform")

NUMBER_OF_SEARCHES = 3

planner_agent = Agent(
    name="Planner Agent",
    instructions=f"You are a helpful research agent. Given a query, you come up with a set of web searches to perform to best answer the query. Output {NUMBER_OF_SEARCHES} terms to search for.",
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

searches = await Runner.run(planner_agent, "Best Agentic AI frameworks in 2025.")

print(searches.final_output_as(WebSearchPlan))

In [ ]:
search_agent = Agent(
    name="Search Agent",
    instructions="You are a helpful research agent. You are given a list of web searches to perform. You perform the searches and produce a concise summary of results. Capture the main points. This will be consumed by someone synthesizing the report so it is vital to capture the essence.",
    model="gpt-4o-mini",
    tools=[WebSearchTool(search_context_size="low")],
    model_settings=ModelSettings(tool_choice="required")
)

search_results = await Runner.run(search_agent, "best Agentic AI frameworks 2025")

print(search_results.final_output)

In [ ]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The final report")
    follow_up_questions: list[str] = Field(description="Suggested topics to research further")

writer_agent = Agent(
    name="Writer Agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData
)

writer_output = await Runner.run(writer_agent, str(search_results.final_output))
print(writer_output.final_output_as(ReportData))

In [ ]:
@function_tool
def send_email(subject: str, html_body: str):
    """Send an email with the given subject and HTML body.
    
    Args:
        subject: The subject of the email
        html_body: The HTML body of the email
    """
    print(f"Sending email with subject: {subject}")
    print(f"HTML body: {html_body}")
    return "Email sent successfully"


INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

email_output = await Runner.run(email_agent, writer_output.final_output_as(ReportData).markdown_report)
print(email_output.final_output)

In [ ]:
planner_agent_tool = planner_agent.as_tool(
    tool_name="research_planner_tool",
    tool_description="Tool for planning the research process"
)

search_agent_tool = search_agent.as_tool(
    tool_name="search_tool",
    tool_description="Tool for performing web searches"
)

writer_agent_tool = writer_agent.as_tool(
    tool_name="writer_tool",
    tool_description="Tool for writing the final report"
)

email_agent_tool = email_agent.as_tool(
    tool_name="email_tool",
    tool_description="Tool for sending the final report via email"
)

In [ ]:
orchestrator_agent = Agent(
    name="Research Orchestrator",
    instructions="""
    You are a helpful research assistant. You are given a research query. Based on the query, you perform the following steps:
    1. Plan the research process
    2. Perform the web searches
    3. Write the final report
    4. Send the final report via email
    You do not perform the steps yourself, but you use the tools provided to you to perform the steps.
    """,
    model="gpt-4o-mini",
    tools=[
        planner_agent_tool,
        search_agent_tool,
        writer_agent_tool,
        email_agent_tool
    ]
)
orchestrator_output = await Runner.run(orchestrator_agent, "Best Agentic AI frameworks in 2025.")
print(orchestrator_output.final_output)